In [61]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Softmax, Input, BatchNormalization, Dropout

from sklearn.model_selection import train_test_split

import tensorflow as tf
from load_policy import load_policy
import pickle
import numpy as np
import gym

import tf_util


In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [53]:
expert_policy_file = "experts/Hopper-v1.pkl"
env_name = "Hopper-v1"
rounds = 1000
max_steps = 1000

In [54]:
data = pickle.load(open(expert_policy_file, "rb"))
policy_fn = load_policy(expert_policy_file)

obs (1, 11) (1, 11)


In [55]:
observations = []
actions = []
rewards = []

with tf.Session() as sess:
#     tf_util.initialize()
    env = gym.make(env_name)
    for i in range(rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = policy_fn(obs[None,:])
            observations.append(obs)
            actions.append(action)
            obs, r_, done, _ = env.step(action)
            rewards.append(r_)
            r += r_
            if done:
                break
            

[2018-06-06 01:31:39,553] Making new env: Hopper-v1


In [63]:
observations = np.array(observations)
actions = np.vstack(actions)

obs_train, obs_valid, act_train, act_valid = \
    train_test_split(observations, actions, test_size = 0.2, random_state = 1998)

In [171]:
input_dim = observations.shape[-1]
bc = Sequential()
bc.add(Dense(32, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
bc.add(Dense(64, input_dim=input_dim, kernel_initializer='normal', activation='relu'))

bc.add(Dense(256, kernel_initializer='normal', activation='relu'))
# bc.add(BatchNormalization())
# bc.add(Dropout(0.5))
bc.add(Dense(64, kernel_initializer='normal', activation='relu'))

bc.add(Dense(3, kernel_initializer='normal'))


In [172]:
bc.compile(optimizer='adam',loss='mse', metrics=['accuracy', 'mae'])

In [173]:
bc.fit(obs_train, act_train, validation_data=[obs_valid, act_valid],  epochs=15, batch_size= 512)

Train on 800000 samples, validate on 200000 samples
Epoch 1/15
800000/800000 [==============================] - 4s 5us/step - loss: 0.0696 - acc: 0.9274 - mean_absolute_error: 0.1096 - val_loss: 0.0043 - val_acc: 0.9708 - val_mean_absolute_error: 0.0456
Epoch 2/15
800000/800000 [==============================] - 3s 4us/step - loss: 0.0024 - acc: 0.9769 - mean_absolute_error: 0.0342 - val_loss: 0.0017 - val_acc: 0.9804 - val_mean_absolute_error: 0.0286
Epoch 3/15
800000/800000 [==============================] - 3s 4us/step - loss: 0.0014 - acc: 0.9820 - mean_absolute_error: 0.0268 - val_loss: 0.0011 - val_acc: 0.9850 - val_mean_absolute_error: 0.0238
Epoch 4/15
800000/800000 [==============================] - 3s 4us/step - loss: 0.0011 - acc: 0.9840 - mean_absolute_error: 0.0236 - val_loss: 0.0011 - val_acc: 0.9845 - val_mean_absolute_error: 0.0226
Epoch 5/15
800000/800000 [==============================] - 3s 4us/step - loss: 9.1688e-04 - acc: 0.9855 - mean_absolute_error: 0.0215 - val

In [92]:
sum(rewards)/1000

3777.9943277272437

In [174]:
o = np.random.random(11)[None, :]
bc.predict(o)

array([[-2.9968495,  2.3936622,  2.0564926]], dtype=float32)

In [175]:
with tf.Session():
    print(policy_fn(o))

[[-2.602352  -4.7028055  2.0624363]]


In [176]:
m_rewards = []

def run_model(m, m_rounds, max_steps):
    env = gym.make(env_name)
    for i in range(m_rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = m.predict(obs[None,:])
#             observations.append(obs)
#             actions.append(action)
            obs, r_, done, _ = env.step(action)
            m_rewards.append(r_)
#             r += r_
            if done:
                break

In [179]:
run_model(bc, 1000, 1000)

[2018-06-06 02:30:45,282] Making new env: Hopper-v1


In [180]:
sum(m_rewards)/1000

3525.8924162780386

In [181]:
len(m_rewards)

968356